<a href="https://colab.research.google.com/github/Hagurou/ML-algo/blob/main/DecisionTreeID3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from typing import List, Optional

In [ ]:
def entropy(freq: np.ndarray) -> float:
    freq = freq[freq > 0]
    if len(freq) == 0:
        return 0.0
    probs = freq / freq.sum()
    return -np.sum(probs * np.log2(probs))

In [ ]:
class TreeNode:
  def __init__(self,ids: List[int],entropy: float,depth: int,children: Optional[List["TreeNode"]] = None,split_attribute: Optional[str] = None,order: Optional[List[str]] = None,label: Optional[str] = None):
    self.ids = ids
    self.entropy = entropy
    self.depth = depth
    self.split_attribute = split_attribute
    self.children = children or []
    self.order = order
    self.label = label

In [ ]:
class DecisionTreeID3:
  def __init__(self, max_depth: int = 10, min_samples_split: int = 2, min_gain: float = 1e-4):
   self.root = None
   self.max_depth = max_depth
   self.min_samples_split = min_samples_split
   self.min_gain = min_gain
   self.data = None
   self.target = None
   self.attributes = None
   self.labels = None
   self.n_train = 0
  def _entropy(self, ids: List[int]) -> float:
        if not ids:
            return 0.0
        freq = np.array(self.target.iloc[ids].value_counts())
        return entropy(freq)

  def _discretize_numeric(self, data: pd.DataFrame, attribute: str, n_bins: int = 3) -> pd.Series:
        if data[attribute].dtype in (np.float64, np.int64):
            return pd.cut(data[attribute], bins=n_bins, labels=False, include_lowest=True).astype(str)
        return data[attribute]

  def _split(self, node: TreeNode) -> List[TreeNode]:
        if len(node.ids) < self.min_samples_split:
            return []

        best_gain = 0
        best_attribute = None
        best_splits = None
        best_order = None
        sub_data = self.data.iloc[node.ids]

        for attribute in self.attributes:
            values = self._discretize_numeric(sub_data, attribute).unique()
            if len(values) <= 1:
                continue

            splits = [sub_data.index[sub_data[attribute] == val].tolist() for val in values]
            splits = [[idx for idx in split if idx in node.ids] for split in splits]

            if any(len(split) < self.min_samples_split for split in splits):
                continue
            conditional_entropy = sum(
                len(split) / len(node.ids) * self._entropy(split) for split in splits
            )
            gain = node.entropy - conditional_entropy

            if gain > best_gain and gain >= self.min_gain:
                best_gain = gain
                best_attribute = attribute
                best_splits = splits
                best_order = values.tolist()

        if best_attribute is None:
            return []

        node.split_attribute = best_attribute
        node.order = best_order
        return [
            TreeNode(ids=split, entropy=self._entropy(split), depth=node.depth + 1)
            for split in best_splits
        ]

  def _set_label(self, node: TreeNode) -> None:
        if not node.ids:
            node.label = self.labels[0]
        else:
            node.label = self.target.iloc[node.ids].mode()[0]

  def fit(self, data: pd.DataFrame, target: pd.Series) -> None:
        self.n_train = len(data)
        self.data = data
        self.attributes = data.columns.tolist()
        self.target = target
        self.labels = target.unique()

        ids = list(range(self.n_train))
        self.root = TreeNode(ids=ids, entropy=self._entropy(ids), depth=0)
        queue = [self.root]

        while queue:
            node = queue.pop(0)
            if node.depth >= self.max_depth or node.entropy == 0:
                self._set_label(node)
                continue

            node.children = self._split(node)
            if not node.children:
                self._set_label(node)
            else:
                queue.extend(node.children)

  def predict(self, data: pd.DataFrame) -> List[str]:
        predictions = []
        for _, row in data.iterrows():
            node = self.root
            while node.children:
                value = str(row[node.split_attribute])
                if value not in node.order:
                    break
                node = node.children[node.order.index(value)]
            predictions.append(node.label)
        return predictions

In [ ]:
if __name__ == "__main__":
    data = pd.DataFrame({
        'outlook': ['sunny', 'sunny', 'overcast', 'rain', 'rain'],
        'temperature': [85, 80, 83, 70, 68],
        'humidity': ['high', 'high', 'high', 'normal', 'normal'],
        'wind': ['weak', 'strong', 'weak', 'weak', 'strong']
    })
    target = pd.Series(['no', 'no', 'yes', 'yes', 'yes'])

    tree = DecisionTreeID3(max_depth=3, min_samples_split=2, min_gain=1e-4)
    tree.fit(data, target)
    predictions = tree.predict(data)
    print("Predictions:", predictions)

Predictions: ['no', 'no', 'no', 'yes', 'yes']
